In [ ]:
%matplotlib inline

import os

# let's not pollute this blog post with warnings
from warnings import filterwarnings
filterwarnings('ignore')

import keras
import numpy as np
import pandas as pd
import skvideo.io as skv
from tqdm import tqdm

In [ ]:
# load the data
run = '-inception-lrcn-color-micro'
labelpath = os.path.join('train_labels.csv')
train_labels = pd.read_csv(labelpath, index_col='filename')

In [ ]:
train_labels.head()

In [ ]:
train_labels.info()

In [ ]:
train_labels.sum(axis=0).sort_values(ascending=False)

In [ ]:
(train_labels.sum(axis=1) > 1).sum()

In [ ]:
from primatrix_dataset_utils import Dataset

In [ ]:
datapath = os.path.join('.')
whichset = run.split('-')[-1]
redframes = whichset == 'nano'
data = Dataset(datapath=datapath, 
               dataset_type=whichset,
               reduce_frames=redframes, 
               batch_size=32, 
               test=False)

In [ ]:
data.num_classes

In [ ]:
data.num_frames

In [ ]:
data.batch_size

In [ ]:
data.num_samples

In [ ]:
data.y_train.shape[0]

In [ ]:
data.y_val.shape[0]

In [ ]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

print(get_available_gpus())

In [ ]:
from keras.models import Sequential
from keras.layers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.callbacks import ModelCheckpoint
from keras.layers.convolutional import MaxPooling2D, Convolution2D, AveragePooling2D
from keras.layers import Input, Dropout, Dense, Flatten, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import concatenate
from keras import regularizers
from keras import initializers
from keras.models import Model
# Backend
from keras import backend as K
# Utils
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file


#########################################################################################
# Implements the Inception Network v3 (http://arxiv.org/pdf/1602.07261v1.pdf) in Keras. #
#########################################################################################

def conv2d_bn(x, nb_filter, num_row, num_col,
              padding='same', strides=(1, 1), use_bias=False):
    """
    Utility function to apply conv + BN. 
    (Slightly modified from https://github.com/fchollet/keras/blob/master/keras/applications/inception_v3.py)
    """
    channel_axis = -1
    x = TimeDistributed(Convolution2D(nb_filter, (num_row, num_col),
                      strides=strides,
                      padding=padding,
                      use_bias=use_bias,
                      kernel_initializer=initializers.he_normal()))(x)
    x = TimeDistributed(BatchNormalization(axis=channel_axis, scale=False))(x)
    x = TimeDistributed(Activation('relu'))(x)
    return x


def block_inception_a(input):
    # 30 x 7 x 7 x 256
    channel_axis = -1
    
    
    branch_0 = conv2d_bn(input, 64, 1, 1)
    # 30 x 7 x 7 x 96

    branch_1 = conv2d_bn(input, 64, 1, 1)
    # 30 x 7 x 7 x 64
    
    branch_1 = conv2d_bn(branch_1, 64, 3, 3)
    # 30 x 7 x 7 x 96

    branch_2 = conv2d_bn(input, 64, 1, 1)
    # 30 x 7 x 7 x 64
    branch_2 = conv2d_bn(branch_2, 64, 3, 3)
    # 30 x 7 x 7 x 96
    branch_2 = conv2d_bn(branch_2, 64, 3, 3)
    # 30 x 7 x 7 x 96

    branch_3 = TimeDistributed(AveragePooling2D((3,3), strides=(1,1), padding='same'))(input)
    branch_3 = conv2d_bn(branch_3, 64, 1, 1)
    # 30 x 7 x 7 x 96

    x = concatenate([branch_0, branch_1, branch_2, branch_3], axis=channel_axis)
    # 30 x 7 x 7 x 256
    return x


def block_reduction_a(input):
    # 30 x 7 x 7 x 256
    channel_axis = -1

    branch_0 = conv2d_bn(input, 128, 3, 3, strides=(2,2), padding='valid')
    # 30 x 3 x 3 x 128

    branch_1 = conv2d_bn(input, 96, 1, 1)
    # 30 x 7 x 7 x 192
    branch_1 = conv2d_bn(branch_1, 112, 3, 3)
    # 30 x 7 x 7 x 224
    branch_1 = conv2d_bn(branch_1, 128, 3, 3, strides=(2,2), padding='valid')
    # 30 x 3 x 3 x 128

    branch_2 = TimeDistributed(MaxPooling2D((3,3), strides=(2,2), padding='valid'))(input)
     # 30 x 3 x 3 x 256

    x = concatenate([branch_0, branch_1, branch_2], axis=channel_axis)
    return x


def block_inception_b(input):
    channel_axis = -1
    #3 x 3 x 512
    branch_0 = conv2d_bn(input, 192, 1, 1)

    branch_1 = conv2d_bn(input, 96, 1, 1)
    branch_1 = conv2d_bn(branch_1, 112, 1, 3)
    branch_1 = conv2d_bn(branch_1, 128, 3, 1)

    branch_2 = conv2d_bn(input, 96, 1, 1)
    branch_2 = conv2d_bn(branch_2, 96, 3, 1)
    branch_2 = conv2d_bn(branch_2, 112, 1, 3)
    branch_2 = conv2d_bn(branch_2, 112, 3, 1)
    branch_2 = conv2d_bn(branch_2, 128, 1, 3)

    branch_3 = TimeDistributed(AveragePooling2D((3,3), strides=(1,1), padding='same'))(input)
    branch_3 = conv2d_bn(branch_3, 64, 1, 1)

    x = concatenate([branch_0, branch_1, branch_2, branch_3], axis=channel_axis)
    return x


def block_reduction_b(input):
    channel_axis = -1

    branch_0 = conv2d_bn(input, 192, 1, 1)
    branch_0 = conv2d_bn(branch_0, 192, 3, 3, strides=(3, 3), padding='valid')

    branch_1 = conv2d_bn(input, 256, 1, 1)
    branch_1 = conv2d_bn(branch_1, 256, 1, 3)
    branch_1 = conv2d_bn(branch_1, 320, 3, 1)
    branch_1 = conv2d_bn(branch_1, 320, 3, 3, strides=(3,3), padding='valid')

    branch_2 = TimeDistributed(MaxPooling2D((3, 3), strides=(3, 3), padding='valid'))(input)

    x = concatenate([branch_0, branch_1, branch_2], axis=channel_axis)
    return x


def block_inception_c(input):
    channel_axis = -1

    branch_0 = conv2d_bn(input, 256, 1, 1)

    branch_1 = conv2d_bn(input, 192, 1, 1)
    branch_10 = conv2d_bn(branch_1, 128, 1, 3)
    branch_11 = conv2d_bn(branch_1, 128, 3, 1)
    branch_1 = concatenate([branch_10, branch_11], axis=channel_axis)


    branch_2 = conv2d_bn(input, 192, 1, 1)
    branch_2 = conv2d_bn(branch_2, 224, 3, 1)
    branch_2 = conv2d_bn(branch_2, 256, 1, 3)
    branch_20 = conv2d_bn(branch_2, 128, 1, 3)
    branch_21 = conv2d_bn(branch_2, 128, 3, 1)
    branch_2 = concatenate([branch_20, branch_21], axis=channel_axis)

    branch_3 = TimeDistributed(AveragePooling2D((3, 3), strides=(1, 1), padding='same'))(input)
    branch_3 = conv2d_bn(branch_3, 256, 1, 1)

    x = concatenate([branch_0, branch_1, branch_2, branch_3], axis=channel_axis)
    return x


def inception_v3_base(input):
    channel_axis = -1

    # Input Shape is 299 x 299 x 3 (th) or 3 x 299 x 299 (th)
    # Input Shape is 30 x 64 x 64 x 3
    net = conv2d_bn(input, 32, 4, 4, strides=(2,2), padding='valid')
    # 30 x 31 x 31 x 32
    net = conv2d_bn(net, 32, 3, 3)
    # 30 x 31 x 31 x 32
    net = conv2d_bn(net, 64, 3, 3)
    # 30 x 31 x 31 x 64

    
    branch_0 = TimeDistributed(MaxPooling2D((3,3), strides=(2,2), padding='valid'))(net)
    # 30 x 15 x 15 x 64
    
    branch_1 = conv2d_bn(net, 96, 3, 3, strides=(2,2), padding='valid')
    # 30 x 15 x 15 x 96
    
    net = concatenate([branch_0, branch_1], axis=channel_axis)
    # 30 x 15 x 15 x 160
    
    branch_0 = conv2d_bn(net, 64, 1, 1)
    # 30 x 15 x 15 x 64
    branch_0 = conv2d_bn(branch_0, 64, 3, 3)
    # 30 x 15 x 15 x 64

    branch_1 = conv2d_bn(net, 64, 1, 1)
    # 30 x 15 x 15 x 64    
    branch_1 = conv2d_bn(branch_1, 64, 1, 5)
    # 30 x 15 x 15 x 64    
    branch_1 = conv2d_bn(branch_1, 64, 5, 1)
    # 30 x 15 x 15 x 64    
    branch_1 = conv2d_bn(branch_1, 64, 3, 3)
    # 30 x 15 x 15 x 64

    net = concatenate([branch_0, branch_1], axis=channel_axis)
    # 30 x 15 x 15 x 128

    branch_0 = conv2d_bn(net, 128, 3, 3, strides=(2,2), padding='valid')
    # 30 x 7 x 7 x 128
    
    branch_1 = TimeDistributed(MaxPooling2D((3,3), strides=(2,2), padding='valid'))(net)
    # 30 x 7 x 7 x 128

    net = concatenate([branch_0, branch_1], axis=channel_axis)
    # 30 x 7 x 7 x 256

    
    # 4 x Inception-A blocks
    for idx in range(4):
        net = block_inception_a(net)

    # 30 x 7 x 7 x 256
    # Reduction-A block
    net = block_reduction_a(net)

    # 3 x 3 x 512
    # 7 x Inception-B blocks
    for idx in range(7):
        net = block_inception_b(net)

    # 3 x 3 x 512
    # Reduction-B block
    net = block_reduction_b(net)

    # 1 x 1 x 1024
    # 3 x Inception-C blocks
    #for idx in range(3):
    #    net = block_inception_c(net)
    # 1 x 1 x 1024
    return net


def inception_v3(num_classes=24, dropout_keep_prob=0.5, include_top=True):
    '''
    Creates the inception v3 network
    Args:
    	num_classes: number of classes
    	dropout_keep_prob: float, the fraction to keep before final layer.
    
    Returns: 
    	logits: the logits outputs of the model.
    '''

    inputs = Input((data.num_frames, data.width, data.height, 3))
    x = BatchNormalization(axis=-1)(inputs)

    # Make inception base
    x = inception_v3_base(x)


    # Final pooling and prediction
    if include_top:
        # 30 x 1 x 1 x 1024
        x = TimeDistributed(Flatten())(x)
        x = LSTM(256, return_sequences=False)(x)
        # 30 x 256
        #x = LSTM(64, return_sequences=False)(x)
        # 256
        x = Dropout(dropout_keep_prob)(x)
        
        # 64
        x = Dense(units=num_classes, activation='sigmoid')(x)

    model = Model(inputs, x, name='inception_v3')
    return model


In [ ]:
# instantiate model
model = inception_v3()


# classifier with sigmoid activation for multilabel
model.add(Dense(data.num_classes, activation='softmax'))

adam = keras.optimizers.Adam(lr=0.001, decay=0.0, clipnorm=5.)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=20, min_lr=0.00001, verbose=True)


# compile the model with binary_crossentropy loss for multilabel
model.compile(optimizer=adam, loss='binary_crossentropy')

model_name = 'model' + run + '.h5'
checkpoint = ModelCheckpoint(model_name, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

callbacks_list = [checkpoint, reduce_lr]


# look at the params before training
model.summary()

In [ ]:
model.fit_generator(
    data.batches(), 
    steps_per_epoch=int(data.num_batches/20),                  # data.num_batches to train on full set 
    epochs=6000, 
    validation_data=data.val_batches(), 
    validation_steps=int(data.num_val_batches/20),                  # data.num_val_batches to validate on full set
    callbacks=callbacks_list
)

In [ ]:
# load model
from keras.models import load_model

trained_model = load_model(model_name)

# generate predictions
for batch_num in tqdm(range(data.num_test_batches), total=data.num_test_batches):

    # make predictions on batch
    results = trained_model.predict(next(data.test_batches()), 
                                          batch_size=data.batch_size, 
                                          verbose=0)

    # update submission format dataframe stored in dataset object
    data.update_predictions(results)          
    

In [ ]:
data.predictions.to_csv(os.path.join(data.datapath, 'predictions' + run + '.csv'))